In [ ]:
!sudo apt update && sudo apt install --yes python3 python3-pip libsnappy-dev && \
pip3 install kafka-python lz4 python-snappy crc32c

In [5]:
import numpy as np

def data_generator():
    a = np.random.normal(loc=1, scale=2)
    b = np.random.normal(loc=-1, scale=2)
    c = np.random.normal(loc=5, scale=2)
    x1 = np.random.randint(-100, 100)
    x2 = np.random.randint(-100, 100)
    x3 = np.random.randint(-100, 100)
    return {'X': [2 * (a + x1), 3 * (b + x2), x3], 'y': x1 + x2 + x3}

data_generator()

{'X': [0.015387396229171069, -231.31426163510656, 66], 'y': -11}

In [2]:
bootstrap_server = ['...:9092']
topic = 'train'
group_id = '1'

In [3]:
from typing import Dict, NamedTuple
import logging
import datetime
import json
from collections import namedtuple

import kafka


class RecordMetadata(NamedTuple):
    topic: str
    partition: int
    offset: int



def generate(n):

    producer = kafka.KafkaProducer(
        bootstrap_servers=bootstrap_server,
        security_protocol="PLAINTEXT",
        # sasl_mechanism="SCRAM-SHA-512",
        # sasl_plain_username=user,
        # sasl_plain_password=password,
        # ssl_cafile="/usr/local/share/ca-certificates/Yandex/YandexInternalRootCA.crt",
        value_serializer=serialize,
    )

    try:
        for i in range(n):
            record_md = send_message(producer, topic)
            print(
                f"Msg sent. Topic: {record_md.topic}, partition:{record_md.partition}, offset:{record_md.offset}"
            )
    except kafka.errors.KafkaError as err:
        logging.exception(err)
    producer.flush()
    producer.close()


def send_message(producer: kafka.KafkaProducer, topic: str) -> RecordMetadata:
    data = data_generator()
    future = producer.send(
        topic=topic,
        key=str(np.random.randint(0, 2)).encode("ascii"),
        value=data,
    )

    # Block for 'synchronous' sends
    record_metadata = future.get()
    return RecordMetadata(
        topic=record_metadata.topic,
        partition=record_metadata.partition,
        offset=record_metadata.offset,
    )


def data_generator():
    a = np.random.normal(loc=1, scale=2)
    b = np.random.normal(loc=-1, scale=2)
    c = np.random.normal(loc=5, scale=2)
    x1 = np.random.randint(-100, 100)
    x2 = np.random.randint(-100, 100)
    x3 = np.random.randint(-100, 100)
    return {'X': [2 * (a + x1), 3 * (b + x2), x3], 'y': x1 + x2 + x3}


def serialize(msg: Dict) -> bytes:
    return json.dumps(msg).encode("utf-8")


In [6]:
generate(10)

Msg sent. Topic: train, partition:0, offset:30
Msg sent. Topic: train, partition:0, offset:31
Msg sent. Topic: train, partition:0, offset:32
Msg sent. Topic: train, partition:0, offset:33
Msg sent. Topic: train, partition:0, offset:34
Msg sent. Topic: train, partition:0, offset:35
Msg sent. Topic: train, partition:0, offset:36
Msg sent. Topic: train, partition:0, offset:37
Msg sent. Topic: train, partition:0, offset:38
Msg sent. Topic: train, partition:0, offset:39
